<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>
<br>
<h1 align=center><font size = 5>IBM Data Science Capstone - Eugene Bible</font></h1>
<h2 align=center>Week 3 - Segmenting and Clustering Neighborhoods in Toronto<h2>

# Introduction

This is the Jupyter Notebook to be used for the Week 3 project in the IBM Data Science Capstone course of the IBM Data Science Professional Certificate program by Coursera.

In this Jupyter notebook, neighborhoods in the city of Toronto will be explored, segmented, and clustered. 
First, the data will be scraped from the Wikipedia page and be wrangled, cleaned, and then read into a pandas dataframe in a structured format. Once structured, the neighborhoods in Toronoto will be explored and clustered.

Author: Eugene Bible

This project link on IBM Watson Studio: https://jp-tok.dataplatform.cloud.ibm.com/analytics/notebooks/v2/007ab5db-57d4-4ac7-b5fd-08ab9d2e0afc/view?projectid=0e052036-fd3e-4893-a04b-46349d1af3d9&context=analytics

This project link on Github: https://github.com/Glitch852/Coursera_Capstone/blob/master/IBM%20Data%20Science%20Capstone%20-%20EB.ipynb

## Table of Contents
<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>

[1. Scraping the Data and Creating the Dataframe](#1.-Scraping-the-Data-and-Creating-the-Dataframe) <br>
[2. Exploring and Clustering the Groups of Toronto](#2.-Exploring-and-Clustering-the-Groups-of-Toronto)<br>
[3. Clustering the Neighborhoods with K-Means](#3.-Clustering-the-Neighborhoods-with-K\-Means)<br>
[4. Summary and Conclusion](#4.-Summary-and-Conclusion)


</font>
</div>

Import the necessary libraries:

In [96]:
import urllib.request,urllib.parse,urllib.error # Library to parse websites
from bs4 import BeautifulSoup # Library to handle web scraping

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder for getting lat/lon

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import json # library to handle json files

import requests #library to handle requests
from pandas.io.json import json_normalize #transform JSON file into pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


# 1. Scraping the Data and Creating the Dataframe

### Scrape the Wikipedia page to obtain the table of postal codes/boroughs/neighborhoods

Page found at: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [97]:
# Define the URL of the webpage to scrape
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Get HTML
html = urllib.request.urlopen(url).read()
#print(html)

# Use BeautifulSoup to parse the html
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

# Parse the table only and pull all rows
table = soup.find('table')
#print(table.prettify())

### Transform the table into a Pandas Dataframe
Requirements:
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [98]:
# Scraping the rows of the table
table_rows = table.find_all('tr')

rows = []
for tr in table_rows:
    #print('count = ' + str(count))
    th = tr.find_all('th') # Grab header
    td = tr.find_all('td') # Grab row
    if th:
        row = [th.text.rstrip() for th in th]
        rows.append(row)
    else:
        row = [tr.text.rstrip() for tr in td]
        if row[1] == 'Not assigned': continue # If no borough, don't keep the row
        if row[2] == 'Not assigned': row[2] = row[1] # If neighborhood is not assigned, set it to the Borough
        rows.append(row)  
            
# Assign list of rows to a Pandas DF
nhScraped_df = pd.DataFrame(rows)

# Name the columns after the first row (headers) then drop the header row
nhScraped_df.columns = nhScraped_df.iloc[0]
nhScraped_df = nhScraped_df.drop(nhScraped_df.index[0])
nhScraped_df.head(10)


,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Downtown Toronto,Queen's Park
7,M9A,Queen's Park,Queen's Park
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern
10,M3B,North York,Don Mills North


In [99]:
# Let's check our counts of postal codes, boroughs, and neighborhoods.

print('The dataframe has {} postal codes, {} boroughs, and {} neighborhoods.'.format(
        len(nhScraped_df['Postcode'].unique()),
        len(nhScraped_df['Borough'].unique()),
        len(nhScraped_df['Neighbourhood'].unique())
    )
)

The dataframe has 103 postal codes, 11 boroughs, and 207 neighborhoods.


In [100]:
# Now that we have the 'Not Assigned' entries taken care of, need to combine the rows with the same Postcode but different neighborhoods

# Use groupby to sort the list by Postcode
nhGrouped_df = nhScraped_df.groupby(['Postcode','Borough'])
#print(nhGrouped_df)

# Print to check for proper sorting
for key, item in nhGrouped_df:
    print(nhGrouped_df.get_group(key), "\n\n")

0 Postcode      Borough Neighbourhood
8      M1B  Scarborough         Rouge
9      M1B  Scarborough       Malvern 


0  Postcode      Borough   Neighbourhood
21      M1C  Scarborough  Highland Creek
22      M1C  Scarborough      Rouge Hill
23      M1C  Scarborough      Port Union 


0  Postcode      Borough Neighbourhood
33      M1E  Scarborough     Guildwood
34      M1E  Scarborough   Morningside
35      M1E  Scarborough     West Hill 


0  Postcode      Borough Neighbourhood
39      M1G  Scarborough        Woburn 


0  Postcode      Borough Neighbourhood
43      M1H  Scarborough     Cedarbrae 


0  Postcode      Borough        Neighbourhood
54      M1J  Scarborough  Scarborough Village 


0  Postcode      Borough         Neighbourhood
66      M1K  Scarborough  East Birchmount Park
67      M1K  Scarborough               Ionview
68      M1K  Scarborough          Kennedy Park 


0  Postcode      Borough Neighbourhood
79      M1L  Scarborough      Clairlea
80      M1L  Scarborough   Gold

In [101]:
# Join neighborhoods of the grouped entries (i.e. the groups with the same postcode), separated by commas
nhJoined_df = nhGrouped_df['Neighbourhood'].apply(', '.join)

nhJoined_df = nhJoined_df.reset_index()

nhJoined_df.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [102]:
# Check the number of rows in our df, should still have 103 postal codes, meaning 103 rows now.
nhJoined_df.shape

(103, 3)

### Getting latitude and longitude for each neighborhood
Now that we have a dataframe of the postal codes of each neighborhood, borough name, and neighborhood name, to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [103]:
## Using Python's built-in geocode NOTE: TOO UNRELIABLE. USED THE GEOSPATIAL_DATA PAGE PROVIDED BY INSTRUCTOR.
#
## initialize your variable to None
#lat_lng_coords = None
#
#address = 'M1J Scarborough Scarborough Village'
#geolocator = Nominatim(user_agent="EB_explores_canada")
#
#while(lat_lng_coords is None):
#    lat_lng_coords = geolocator.geocode(address)
#
#print(lat_lng_coords)
#    
#nlat = lat_lng_coords.latitude
#nlon = lat_lng_coords.longitude
#
#
#print(nlat, nlon)

Note: Package was too unreliable to get all lat/lon values. Used the CSV file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [104]:
url = 'http://cocl.us/Geospatial_data'
response = urllib.request.urlopen(url)
latlng_df = pd.read_csv(response)


latlng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
# Change column name so both dataframes have the same column name
latlng_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
latlng_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
# Merge two Dataframes on column 'Postcode'
neighborhoods_df = nhJoined_df.merge(latlng_df, on='Postcode')

neighborhoods_df = neighborhoods_df.rename(columns={'Neighbourhood': 'Neighborhood'}) # Because I'm American. (Sorry.)

neighborhoods_df.head(100)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [ ]:
# Let's check our neighborhood and borough counts. Should still have 103 postal codes and 11 boroughs.
print('The dataframe has {} postal codes and {} boroughs.'.format(
        len(neighborhoods_df['Postcode'].unique()),
        len(neighborhoods_df['Borough'].unique())
    )
)

The dataframe has 103 postal codes and 11 boroughs.


# 2. Exploring and Clustering the Groups of Toronto

### Neighborhood Map Visualization

We will now analyze and group each neighborhood set based on the types of locations within each area. Areas with similar location data (shopping, cafes, parks, etc.) will be grouped into the same group using k-Means.

Let's start by getting a map of the area.

In [ ]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Canada are 43.653963, -79.387207.


I performed this analysis with ONLY boroughs that contain 'Toronto' at first, but then I decided to do a larger analysis for the whole city.

The following commented-out code was needed for the 'Toronto'-only version:

In [ ]:
#toronto_df = neighborhoods_df.loc[(neighborhoods_df['Borough'].str.contains('Toronto'))]

#print(toronto_df)

In [ ]:
# create map of Toronto using latitude and longitude values
#toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11.5)

## add markers to map
#for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
#    label = '{}, {}'.format(neighborhood, borough)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(toronto_map)  
#    
#toronto_map

Analysis re-done with all of Toronto:

In [ ]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_df['Latitude'], neighborhoods_df['Longitude'], neighborhoods_df['Borough'], neighborhoods_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

Define Foursquare credentials and version

In [ ]:
# The code was removed by Watson Studio for sharing.

Your credentials: [hidden for assignment]


### Fetch venues for analysis

Next, let's create a function to get all the venues within 500m of each neighborhood in Toronto.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# Let's call the function and see what kind of venues we get
toronto_venues = getNearbyVenues(names=neighborhoods_df['Neighborhood'],
                                   latitudes=neighborhoods_df['Latitude'],
                                   longitudes=neighborhoods_df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

We have now pulled all venues within 500m of each neighborhood. Let's take a look at the venues.

In [ ]:
#Check number of venues
print(toronto_venues.shape)
toronto_venues.head()

(2213, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


In [ ]:
# Let's check how many entries there are for each neighborhood.
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


In [ ]:
# Check the number of UNIQUE categories
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 unique categories.


### Mathematical Analysis - One hot encoding and grouping

One Hot Encoding - Converting categorical variables into a numerical form for mathematical analysis.

In [ ]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# Let's look at all the different columns
cols = list(toronto_onehot)
#print(cols)

# Move the neighborhood column to the front of the list
cols.insert(0, cols.pop(cols.index('Neighborhood')))
#print(cols)

# Re-order the toronto_onehot df according to the re-ordered columns list
toronto_onehot = toronto_onehot.loc[toronto_onehot.index[:], cols]
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
# Let's confirm that we still have all the entries. (2213 entries with 266 categories)
toronto_onehot.shape

(2213, 266)

Let's group rows by neighborhood and take the mean of the frequency of occurance for each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,

In [ ]:
# Now we've grouped the same neighborhoods together.
toronto_grouped.shape

(101, 266)

 Note: from the 103 postal codes, 2 seem to have had no registered venues, we now only have 101 neighborhood sets.

Print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue','Freq'] # Name the columns 
    temp = temp.iloc[1:] # Drop the top row
    temp['Freq'] = temp['Freq'].astype(float) #Convert frequency values to float
    temp = temp.round({'Freq': 2}) # And round
    print(temp.sort_values('Freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
          Venue  Freq
0   Coffee Shop  0.07
1    Steakhouse  0.04
2          Café  0.04
3           Bar  0.04
4  Burger Joint  0.03


----Agincourt----
                       Venue  Freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2  Latin American Restaurant   0.2
3             Clothing Store   0.2
4               Skating Rink   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               Venue  Freq
0         Playground   0.5
1               Park   0.5
2  Accessories Store   0.0
3        Men's Store   0.0
4  Mobile Phone Shop   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 Venue  Freq
0        Grocery Store   0.2
1          Pizza Place   0.1
2  Fried Chicken Joint   0.1
3          Coffee Shop   0.1
4       Sandwich Place   0.1


----Alderwood, Long Branch----
          Venue  Freq
0   Pizza Place  0.22
1    

### Getting the top venues in each neighborhood

In [ ]:
# Let's create a function to get the most common venue types in each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

We now have the 10 most common venue types in each neighborhood set. Now to group them with K-Means

# 3. Clustering the Neighborhoods with K-Means

### K-Means Clustering

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

Interestingly, it seems as if with 5 clusters as in the New York analysis, most neighborhoods fall into one category. 

Let's create new dataframe that includes cluster labels as well as top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

toronto_merged.head() # check the last columns!

### Map visualization of K-Means clustered Toronto

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can see that most neighborhoods in Toronto are similar (the "red dot" category). Let's examine what each color could possibly mean.

### Examination of Clusters

#### Cluster 1 (Red)

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Looking at the most common venues in the red cluster 1, we can see there are mainly places to eat, with some other places commonly visited in everyday life. Drugstores, coffee shops, and gas stations are all places we frequent in daily life. From this, we can assume that the red neighborhoods are most likely places where people live.

#### Cluster 2 (Purple)

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

For the purple cluster, we can the most common venue is parks. Parks are then followed by places like tennis courts/dance studios, event spaces, convenience stores, and fast food/snack/dessert shops. From this, we can gather that these are more recreational spaces. Places where people congregate for exercise and socializing.

#### Cluster 3 (Light Blue)

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

This cluster seemed to be significantly different from the rest just because of the existance of a gym. It was most likely similar to one of the other neighborhoods, but as a result of our request of 5 clusters, needed to find something different enough to warrant a separate category. The algorithm seems to have used 'Gym' for this.

#### Cluster 4 (Light Green)

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

This was most likely another result of our choice of 5 clusters. This cluster seems to have been differentiated based on it's existance of home repair/construction shops. All 3 boroughs have home repair/construction in the 1st and 2nd most common venue, so was likely separated based on those.

#### Cluster 5 (Orange)

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 5 seems to have been separated due to the existance of the golf courses, which only existed in these areas, meriting these 2 neighborhoods being clustered separately. We can see there were also pools and dog runs in the area. These areas are most likely good exercising/socializing places similar to purple.

## 4. Summary and Conclusion

### Summary

The purpose of this analysis was to group and cluster each neighborhood of Toronto into similar neighborhoods based on the venues that exist within them. Neighborhoods with similar venues were grouped together into a total of 5 clusters:

- Cluster 1 (Red): Easy living - Restaurants, convenience stores and drug stores, cafes, gas stations, and bars. Places where people live.
- Cluster 2 (Purple): Socializing and outdoors - Parks, tennis courts, event spaces, and cafes/donut shops. Places where people go for socializing and exercise.
- Cluster 3 (Light Blue): Gym! - With only 4 clusters, this most likely would have been purple.
- Cluster 4 (Light Green): Home Care -  Construction, things for home repair and maintenance, and shops seem to be the theme.
- Cluster 5 (Orange): Exercise and Socializing - Similar to purple, these areas contained golf courses, swimming pools, and dog runs. Good for exercise! 

### K-Means

In our K-Means analysis I chose to use 5 clusters, as was done in the analysis of New York City. Because of the overwhelming number of neighborhoods placed into the same cluster (red), we probably could have used more clusters for a more accurate and deeper analysis. Splitting up those red clusters further might have gotten more varied results with more distinctive neighborhoods.

### Closing Remarks

I learned more on this one project than I did in all the other labs put together. Having to actually do something for yourself, rather than just run cells that are mostly completed forced me to spend most of my time just learning what exactly each line of code does and how to use it. The analysis performed could probably have been done cleaner and more efficiently, and I think there is still some refinement that could be done with the number of clusters, but overall I am happy with the result, which seemed reasonable. 

Thank you to the professor and staff for the great course and to you, the reader, for reading my project report!